The way Emily gets this to work is:

In the bash console, type `\ArcGIS\Pro\bin\Python\Scripts\proenv` to activate the ArcGIS Pro Python 3 conda environment. This is the only way I know to access ArcPy outside of an ArcGIS GUI. Change directories into the location of this code: `cd Code\beachplant_dataextraction` Then open a jupyter lab session: `jupyter lab`. At the beginning, you may need to install some additional Python packages, such as jupyterlab (`conda install -c conda-forge jupyterlab`). 

## First, import the necessary modules

In [1]:
import os
import arcpy
import time
import pandas as pd
import sys
import numpy as np
import pyproj

# Set up alert sound for long-running cells
from IPython.display import Audio
sound_file = r'\\Mac\stor\Code\van-sliding-door-daniel_simon.wav'

In [2]:
print("Date: {}".format(datetime.date.today()))
print('pandas version: {}'.format(pd.__version__))
print('numpy version: {}'.format(np.__version__))
print('pyproj version: {}'.format(pyproj.__version__))

Date: 2019-07-16
pandas version: 0.20.1
numpy version: 1.11.2
pyproj version: 1.9.5.1


## Set up arcpy environments

In [196]:
# Set year of analysis
year = 2008

# Paths
home = r'\\Mac\stor\Projects\SA_DataProcessing\GDBs'
data_dir = r'\\Mac\stor\Projects\SA_DataProcessing\input_data'

# Set up workspace
yeargdb = os.path.join(home, str(year) +'.gdb')
if not os.path.exists(yeargdb):
    arcpy.CreateFileGDB_management(home, str(year))
    print('Created new gdb "{}.gdb"'.format(year))
arcpy.env.workspace = yeargdb
arcpy.env.scratchWorkspace = home
arcpy.env.overwriteOutput = True 
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

## Initialize filenames and other variables

In [197]:
# Files
DEM_dir = os.path.join(data_dir, 'ASIS_Lidar2008')
DEM = os.path.join(arcpy.env.workspace, 'DEM08')
DEM = 'DEM08'
veg_polyshp = os.path.join(data_dir, 'ASIS_veghabMaps', 'asis_pipl_habitat2008.shp')

# Parameters
year = 2008
MHW = 0.34
MLW = -0.13
MTL = (MHW + MLW)/2
gridsz = 5
plants_epsg = 26918
fill = -99999
ranPts_per_PlantPt = 3
dem_null = -32767

# Initialize
crs = arcpy.SpatialReference(plants_epsg)
arcpy.env.outputCoordinateSystem = crs

# Output filenames
inletLines = 'inletLines'
SLpts = os.path.join(yeargdb, 'SLpts')
DLpts = os.path.join(yeargdb, 'DLpts')
DHpts = os.path.join(yeargdb, 'DHpts')
barrierBoundary = 'bndpoly08_toSL'
plants_outfc = 'pts_plants'
ranPts = 'pts_random'
trainingPts = 'pts_training'
DEMres = '{}_{}m'.format(DEM, gridsz)
veg_fc = 'asis_pipl_habitat2008'
vegRaster = 'asis_pipl_habitat2008_rst'

## Define functions

These were copied from bi-transect-extractor and in some cases modified.

In [5]:
def RasterToLandPerimeter(in_raster, out_polygon, threshold, agg_dist='10 METERS', min_area='300 SquareMeters',
    min_hole_sz='300 SquareMeters', manualadditions=None):
    """
    Raster to Polygon: DEM => Reclass => MHW line
    """
    r2p = os.path.join(arcpy.env.scratchGDB, out_polygon+'_r2p_temp')
    r2p_union = os.path.join(arcpy.env.scratchGDB, out_polygon+'_r2p_union_temp')

    # Reclassify the DEM: 1 = land above threshold; the rest is nodata
    rastertemp = arcpy.sa.Con(arcpy.sa.Raster(in_raster)>threshold, 1, None)  # temporary layer classified from threshold
    # Convert the reclass raster to a polygon
    arcpy.RasterToPolygon_conversion(rastertemp, r2p)  # polygon outlining the area above MHW
    if manualadditions: # Manually digitized any large areas missed by the lidar
        arcpy.Union_analysis([manualadditions,r2p], r2p_union, gaps='NO_GAPS')
        arcpy.AggregatePolygons_cartography(r2p_union, out_polygon, agg_dist, min_area, min_hole_sz)
    else:
        arcpy.AggregatePolygons_cartography(r2p, out_polygon, agg_dist, min_area, min_hole_sz)
    print('Aggregation distance: {}\nMinimum area: {}\nMinimum hole size: {}'.format(agg_dist, min_area, min_hole_sz))
    return(out_polygon)

def CombineShorelinePolygons(bndMTL: str, bndMHW: str, inletLines: str,
    ShorelinePts: str, bndpoly: str, SA_bounds: str='', verbose: bool=True):
    """
    Use MTL and MHW contour polygons to create shoreline polygon.
    'Shoreline' = MHW on oceanside and MTL on bayside
    """
    start = time.clock()
    # Inlet lines must intersect the MHW polygon
    symdiff = os.path.join(arcpy.env.scratchGDB, 'shore_1symdiff')
    split = os.path.join(arcpy.env.scratchGDB, 'shore_2split')
    join = os.path.join(arcpy.env.scratchGDB, 'shore_3_oceanMTL')
    erase = os.path.join(arcpy.env.scratchGDB, 'shore_4_bayMTL')
    union_2 = os.path.join(arcpy.env.scratchGDB, 'shore_5union')

    # Create layer (symdiff) of land between MTL and MHW and split by inlets
    print("...delineating land between MTL and MHW elevations...")
    arcpy.Delete_management(symdiff) # delete if already exists
    arcpy.SymDiff_analysis(bndMTL, bndMHW, symdiff)

    # Split symdiff at inlets (and SA_bounds)
    print("...removing the MHW-MTL areas on the oceanside...")
    if len(SA_bounds) > 0:
        arcpy.FeatureToPolygon_management([symdiff, inletLines, SA_bounds], split) # Split MTL features at inlets and study area bounds
    else:
        arcpy.FeatureToPolygon_management([symdiff, inletLines], split) # Split MTL features at inlets
    # Isolate polygons touching shoreline points and erase from symdiff
    arcpy.SpatialJoin_analysis(split, ShorelinePts, split+'_join', "#","KEEP_COMMON", match_option="COMPLETELY_CONTAINS")
    arcpy.Erase_analysis(symdiff, split+'_join', erase)

    # Merge bayside MHW-MTL with above-MHW polygon
    arcpy.Union_analysis([erase, bndMHW], union_2)
    arcpy.Dissolve_management(union_2, bndpoly, multi_part='SINGLE_PART') # Dissolve all features in union_2 to single part polygons
    print('''User input required! Select extra features in {} for deletion.\nRecommended technique: select the polygon/s to keep and then Switch Selection.\n'''.format(os.path.basename(bndpoly)))
    return(bndpoly)

def DEMtoFullShorelinePoly(elevGrid, MTL, MHW, inletLines, ShorelinePts, SA_bounds='', suffix=''):
    """Delinate the full shoreline polygon from the DEM.
    Creates the MTL and MHW contour polygons and then combines them."""
    bndMTL = 'bndpoly_mtl' + suffix
    bndMHW = 'bndpoly_mhw' + suffix
    bndpoly = 'bndpoly' + suffix
    print("Creating the MTL contour polgon from the DEM...")
    RasterToLandPerimeter(elevGrid, bndMTL, MTL, agg_dist='10', min_area='300', min_hole_sz='300')  # Polygon of MTL contour
    print("Creating the MHW contour polgon from the DEM...")
    RasterToLandPerimeter(elevGrid, bndMHW, MHW, agg_dist='10', min_area='300', min_hole_sz='300')  # Polygon of MHW contour
    print("Combining the two polygons...")
    bndpoly = CombineShorelinePolygons(bndMTL, bndMHW, inletLines, ShorelinePts, bndpoly, SA_bounds)
    #DeleteTempFiles()
    return(bndpoly)

def NewBNDpoly(old_boundary, modifying_feature, new_bndpoly='boundary_poly', vertexdist='25 METERS', snapdist='25 METERS', verbose=True):
    """Snaps the boundary polygon to the shoreline points anywhere they don't
    already match and as long as as they are within 25 m of each other."""
    # boundary = input line or polygon of boundary to be modified by newline
    typeFC = arcpy.Describe(old_boundary).shapeType
    if typeFC == "Line" or typeFC =='Polyline':
        arcpy.FeatureToPolygon_management(old_boundary, new_bndpoly, '1 METER')
    else:
        if len(os.path.split(new_bndpoly)[0]):
            path = os.path.split(new_bndpoly)[0]
        else:
            path = arcpy.env.workspace
        arcpy.FeatureClassToFeatureClass_conversion(old_boundary, path, os.path.basename(new_bndpoly))
    typeFC = arcpy.Describe(modifying_feature).shapeType
    if typeFC == "Line" or typeFC == "Polyline":
        arcpy.Densify_edit(modifying_feature, 'DISTANCE', vertexdist)
    arcpy.Densify_edit(new_bndpoly, 'DISTANCE', vertexdist)
    arcpy.Snap_edit(new_bndpoly,[[modifying_feature, 'VERTEX',snapdist]]) # Takes a while
    if verbose:
        print("Created: {} ... Should be in your workspace geodatabase.".format(os.path.basename(new_bndpoly)))
    return new_bndpoly # string name of new polygon

def DFtoFC(df, out_fc, spatial_ref, id_fld='', xy=["seg_x", "seg_y"], keep_fields=[], fill=-99999):
    """Create FC from DF; default only copies X,Y,ID fields
    using too many fields with a large dataset will fail"""
    # Make sure name of index is not also a column name
    print("... converting dataframe to array... ")
    if df.index.name in df.columns:
        df.index.name = 'index'
    # Convert DF to array
    if keep_fields == 'all':
        keep_fields = df.columns
    else:
        keep_fields += xy + [id_fld]
    # First, remove 'object' type columns, all columns not in keep_fields, convert to floats, and fill Nulls.
    # Remove any rows with X == None
    xfld = xy[0]
    df = df[~df[xfld].isnull()]
    df = df[df[xfld]!=fill]
    try:
        arr = (df.select_dtypes(exclude=['object'])
                 .drop(df.columns.drop(keep_fields, errors='ignore'), errors='ignore', axis=1)
                 .astype('f8').fillna(fill).to_records())
    except ValueError:
        df.index.name = 'index'
        arr = (df.select_dtypes(exclude=['object'])
             .drop(df.columns.drop(keep_fields, errors='ignore'), errors='ignore', axis=1)
             .astype('f8').fillna(fill).to_records())
        print('Encountered ValueError while converting dataframe to array so set index name to "index" before running.' )
    # Convert array to FC
    print("... converting array to feature class... ")
    # out_fc = os.path.join(arcpy.env.scratchGDB, os.path.basename(out_fc)) # set out_fc path
    arcpy.Delete_management(out_fc) # delete if already exists
    arcpy.da.NumPyArrayToFeatureClass(arr, out_fc, xy, spatial_ref)
    print()
    return(out_fc)

def CreateShoreBetweenInlets(shore_delineator: str, inletLines: str,
    out_line: str, ShorelinePts: str, proj_code: int=26918, SA_bounds: str='',
    verbose: bool=True, extend_dist='500 Meters', sort_fld=''):
    """Create ShoreBetweenInlets line = oceanside shoreline between inlets;
    Generated from shoreline polygon, inlet lines, and SA bounds;
    Used to produce shoreline points and measure distance to inlet."""
    # initialize temp layer names
    split = os.path.join(arcpy.env.scratchGDB, 'shoreline_split')
    # Ready layers for processing
    typeFC = arcpy.Describe(shore_delineator).shapeType
    if typeFC == "Point" or typeFC =='Multipoint':
        line_temp = os.path.join(arcpy.env.scratchGDB, 'shoreline_line')
        shore_temp = os.path.join(arcpy.env.scratchGDB, 'shoreline_shore')
        # Create shoreline from shoreline points
        arcpy.PointsToLine_management(shore_delineator, line_temp, Sort_Field=sort_fld, Close_Line='NO_CLOSE')
        shore_delineator = shore_temp
        # Merge and then extend shoreline to inlet lines
        arcpy.Merge_management([line_temp,inletLines],shore_delineator)
        arcpy.ExtendLine_edit(shore_delineator, extend_dist)
    # Split shoreline at inlets
    if verbose:
        print("Splitting {} at inlets...".format(os.path.basename(shore_delineator)))
    arcpy.Delete_management(split) # delete if already exists
    arcpy.FeatureToLine_management([shore_delineator, inletLines], split)
    # Delete any lines that are not intersected by a shoreline point.
    if verbose:
        print("Preserving only those line segments that intersect shoreline points...")
    arcpy.SpatialJoin_analysis(target_features=split, join_features=ShorelinePts,
        out_feature_class=split+'_join', join_operation="#", join_type="KEEP_COMMON",
        match_option="COMPLETELY_CONTAINS")
    if not int(arcpy.GetCount_management(split+'_join').getOutput(0)):
        print("PROBABLE ERROR: The output of the spatial join operation is empty. Look at shoreline_split and shoreline_split_join in the scratch gdb.")
    if verbose:
        print("Dissolving the line to create {}...".format(os.path.basename(out_line)))
    dissolve_fld = "FID_{}".format(os.path.basename(shore_delineator))
    arcpy.Dissolve_management(split+'_join', out_line, [[dissolve_fld]], multi_part="SINGLE_PART")
    return out_line

def FCtoDF(fc, xy=False, dffields=[], fill=-99999, id_fld=False, extra_fields=[], verbose=True, fid=False, explode_to_points=False, length=False):
    """Convert FeatureClass to pandas.DataFrame with np.nan values"""
    # 1. Convert FC to Numpy array
    if explode_to_points:
        message = 'Converting feature class vertices to array with X and Y...'
        if not id_fld:
            print('Error: if explode_to_points is set to True, id_fld must be specified.')
        fcfields = [id_fld, 'SHAPE@X', 'SHAPE@Y', 'OID@']
    else:
        fcfields = [f.name for f in arcpy.ListFields(fc)]
        if xy:
            message = 'Converting feature class to array with X and Y...'
            fcfields += ['SHAPE@X','SHAPE@Y']
        else:
            message = '...converting feature class to array...'
        if fid:
            fcfields += ['OID@']
        if length:
            fcfields += ['SHAPE@LENGTH']
    if verbose:
        print(message)
    arr = arcpy.da.FeatureClassToNumPyArray(os.path.join(arcpy.env.workspace, fc),
                    fcfields, null_value=fill, explode_to_points=explode_to_points)
    # 2. Convert array to dict
    if verbose:
        print('...converting array to dataframe...')
    if not len(dffields):
        dffields = list(arr.dtype.names)
    else:
        if xy:
            dffields += ['SHAPE@X','SHAPE@Y']
        if fid:
            dffields += ['OID@']
        if length:
            dffields += ['SHAPE@LENGTH']
    dict1 = {}
    for f in dffields:
        if np.ndim(arr[f]) < 2: # "data must be 1-dimensional" to be included in the DataFrame
            dict1[f] = arr[f]
    # 3. Convert dict to DF
    if not id_fld:
        df = pd.DataFrame(dict1)
    else:
        df = pd.DataFrame(dict1, index=arr[id_fld])
        df.index.name = id_fld
    # replace fill values with NaN values
    df.replace(fill, np.nan, inplace=True) # opposite: df.fillna(fill, inplace=True)
    if len(extra_fields) > 0:
        extra_fields += [x.upper() for x in extra_fields]
        df.drop(extra_fields, axis=1, inplace=True, errors='ignore')
    # restore columns to original order# compare dffields and df.columns. Remove any extra from dffields.
    for f in dffields:
        if not f in df.columns:
            if not f in fcfields:
                print("WARNING: field '{}' listed in dffields is not present in the file.".format(f))
                dffields.remove(f)
    df = df.reindex_axis(dffields, axis=1)
    return(df)

## Pre-process: DEM

In [22]:
# Merge DEM tiles
arcpy.env.workspace = DEM_dir
DEMtiles = arcpy.ListRasters("*","TIF")

tile0 = arcpy.Describe(DEMtiles[0])
tile0.bandCount
tile0.spatialReference
arcpy.GetRasterProperties_management(DEMtiles[0], 'VALUETYPE')
arcpy.GetRasterProperties_management(DEMtiles[0], 'CELLSIZEX')

# Reset workspace
DEM = os.path.join(yeargdb, DEM)
arcpy.CopyRaster_management(DEMtiles[0], DEM)

# Reset workspace
# arcpy.env.workspace = yeargdb

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\DEM08'>

In [23]:
arcpy.Mosaic_management(DEMtiles, DEM, mosaic_type='LAST', nodata_value=dem_null)

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\DEM08'>

In [ ]:
demOut = arcpy.sa.SetNull(DEM, DEM, "VALUE = {}".format(dem_null))
demOut.save(DEM+'_SetNull')

In [ ]:
# Look at environment variables
environments = arcpy.ListEnvironments()
environments.sort(key=str.lower)
for environment in environments:
    env_value = getattr(arcpy.env, environment)
    print("{0:<30}: {1}".format(environment, env_value))

## Pre-process: Shoreline and dune points

Convert spreadsheet of geomorphic feature positions to three point feature classes: SLpts, DLpts, DHpts

In [16]:
# Read spreadsheet
in_xls = os.path.join(data_dir, 'ASIS_shoreNdunes', 'ShoreAndDuneLines_mdva2008Fmod.xlsx')
allpts = pd.read_excel(in_xls)
pts_crs = arcpy.SpatialReference(4269)

# Add sort field to help with PointsToLine later
allpts['sort_fld'] = allpts.sh_segment*1000+allpts.sh_profile

# Create shoreline, dhi, and dlo points FC
DFtoFC(allpts, SLpts, spatial_ref=pts_crs, xy=['sh_long', 'sh_lat'], keep_fields=['sh_segment', 'sort_fld', 'sh_profile', 'sh_ci', 'sh_slope'], fill=fill)

# Create points FC
DFtoFC(allpts, DLpts, spatial_ref=pts_crs, xy=['dl_long', 'dl_lat'], keep_fields=['dl_segment', 'sort_fld', 'dl_profile', 'dl_z', 'dl_mse'], fill=fill)

# Create points FC
DFtoFC(allpts, DHpts, spatial_ref=pts_crs, xy=['dh_long', 'dh_lat'], keep_fields=['dh_segment', 'sort_fld', 'dh_profile', 'dh_z', 'dh_mse'], fill=fill)

... converting dataframe to array... 
... converting array to feature class... 

... converting dataframe to array... 
... converting array to feature class... 

... converting dataframe to array... 
... converting array to feature class... 



'\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\DHpts'

## Shoreline polygon

#### Prep: Look at NPS shoreline polygon first.
Copy NPS shoreline polygon into gdb.

In [26]:
# Look at NPS shoreline polygon
arcpy.env.workspace = yeargdb
bndpoly_in = os.path.join(data_dir, 'ASIS_Boundary', 'Assateague_Island_2003_2010.shp')
bndpoly = 'ASIS_bndpoly03to10'
if not arcpy.Exists(bndpoly):
    arcpy.CopyFeatures_management(bndpoly_in, bndpoly)

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\ASIS_bndpoly03to10'>

#### Create shoreline boundary polygon using MHW on oceanside and MTL on bayside. Split at manually delineated inlet lines.

In [6]:
# Manually create lines that correspond to end of land and cross the MHW line (use bndpoly/DEM)
if not arcpy.Exists(inletLines):
    arcpy.CreateFeatureclass_management(yeargdb, inletLines, 'POLYLINE', spatial_reference=crs)
    print("{} created. Now we'll stop for you to manually create lines at each inlet.".format('inletLines'))

In [ ]:
# Create the shoreline polygon from the DEM and the shoreline points
# Create full barrier shoreline from MHW and MTL.
bndpoly = DEMtoFullShorelinePoly(DEM, MTL, MHW, inletLines, SLpts)
print('Select features from {} that should not be included in the final shoreline polygon. '.format(bndpoly))

# This process can take a while so play sound when cell is complete.
Audio(filename=sound_file, autoplay=True) 

Creating the MTL contour polgon from the DEM...
Aggregation distance: 5
Minimum area: 300
Minimum hole size: 4000
Creating the MHW contour polgon from the DEM...


In [19]:
# Modify the new polygon in two ways: 
#    1. snap the oceanside to the shoreline points, which smooths it and forces it to agree with the more thoroughly-located points. 
#    2. Eliminate all the internal holes in the polygon. 
bndpoly_SLsnap = 'bndpoly08_toSL' #'bndpoly2_snapped'
bndpoly_filled = 'bndpoly_3filled'
bndpoly_multi = 'bndpoly_4multi'

# Snap bndpoly to SL points anywhere they don't already match and as long as as they are within 25 m of each other.
NewBNDpoly(bndpoly, SLpts, bndpoly_SLsnap, '25 METERS', '50 METERS')

# Eliminate any holes in the polygons and make them a single multi-part feature
arcpy.EliminatePolygonPart_management(bndpoly_SLsnap, bndpoly_filled, condition='PERCENT', part_area_percent=99, part_option='CONTAINED_ONLY')
arcpy.Dissolve_management(bndpoly_filled, bndpoly_multi, multi_part='MULTI_PART')

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\bndpoly_4multi'>

## Create raster for each variable

Processing notes:

- Make all subsequent rasters align to the resampled DEM.

#### Set up processing environment (extent, mask, snapRaster)

In [6]:
# Set extent environment to the input DEM
arcpy.env.workspace = yeargdb
arcpy.env.extent = bndpoly_multi
arcpy.env.mask = bndpoly_multi

In [39]:
# Resample to desired analysis size
arcpy.Resample_management(DEM, DEMres, gridsz, 'BILINEAR')

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\DEM08_5m'>

In [7]:
# Set Snap Raster environment. All subsequent rasters will align to these cells.
arcpy.env.snapRaster = DEMres

#### Aspect and slope from DEM

In [40]:
# Make rasters from Elevation
aspect = '{}_aspect'.format(DEMres)
arcpy.Aspect_3d(DEMres, aspect)

slope = '{}_slope'.format(DEMres)
arcpy.Slope_3d(DEMres, slope, 'PERCENT_RISE')

# Play sound when cell is complete.
Audio(filename=sound_file, autoplay=True) 

#### Distances to shoreline, dhi, dlo

In [12]:
# ShoreBetweenInlets
mhw_shoreline = 'SL_line'
CreateShoreBetweenInlets(bndpoly_multi, inletLines, mhw_shoreline, SLpts, plants_epsg)

distSL = 'distSL_SLline'
outDistSL = arcpy.sa.EucDistance(mhw_shoreline, cell_size=gridsz)
outDistSL.save(distSL)

In [273]:
# Make distance rasters
# Distance to dune toes
distDL = 'distDL'
outDistDL = arcpy.sa.EucDistance(DLpts, cell_size=gridsz)
outDistDL.save(distDL)

# Distance to dune crests
distDH = 'distDH'
outDistDH = arcpy.sa.EucDistance(DHpts, cell_size=gridsz)
outDistDH.save(distDH)

# Play sound when cell is complete.
Audio(filename=sound_file, autoplay=True) 

#### Vegetation type

In [44]:
# # Make vegetation raster
if not arcpy.Exists(veg_fc):
    arcpy.CopyFeatures_management(veg_polyshp, veg_fc)
    
arcpy.PolygonToRaster_conversion(veg_fc, 'Veg_Type', vegRaster)

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\asis_pipl_habitat2008_rst'>

#### Distance to inlet
from Ben: "Can you add a distance to inlet field like the plover/barrier data? Although this time, just reference it to ocean city inlet - for plants and randoms. I've attached an edited version of the NPS km markers that they use as reference points. Perhaps the distance to inlet can be aligned with these markers, or these markers serving as reference points instead-distance from the zero marker, along the shoreline path-and according to each successive marker-if this makes sense?"

Ideas for how to execute:

1. create ShoreBetweenInlets
1. convert ShoreBetweenInlets (SL_line) to points or segments
2. add the dist2inlet to each segment
3. for each raster cell: assign the dist2inlet value from the closest segment (Euclidean Allocation)

Use a tool in Arc... Maybe one of these tools is more what Ben is going for anyway. What process are we trying to measure here?

Distance toolset: https://pro.arcgis.com/en/pro-app/tool-reference/spatial-analyst/an-overview-of-the-distance-tools.htm


Cost distance (based on ShoreBetweenInlets) to every segment (above)? or to every cell, but that would add too much cost distance for inland points.
Add vertical restrictions with path distance analysis?

#### Experiment with water level contours

Depth to water - Water level contours

In [36]:
# Convert shapefile to FC in yeargdb
d2w_shp = os.path.join(data_dir, 'Depth2Water', 'base_case_hds', 'WL_Contours.shp')
d2w_fc = 'WL_Contours'
arcpy.FeatureClassToFeatureClass_conversion(d2w_shp, arcpy.env.workspace, d2w_fc)

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\WL_Contours'>

In [45]:
d2w_polys = 'WL_lines2polys'
arcpy.FeatureToPolygon_management(d2w_fc, d2w_polys)

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\WL_lines2polys'>

In [37]:
d2w_topo2rst = 'WL_topo2rst'
arcpy.TopoToRaster_3d(d2w_fc, d2w_topo2rst)

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\WL_topo2rst'>

### Test more complicated distance calculations

Alternatives: 

- dist_sl: distance to filled bndpoly outline
- dist_sl: distance to MHW **line** (ShoreBetweenInlets)
- dist_dh: distance to DH line
- dist_dh: distance to DH line with threshold distance
- dist_dl: distance to DL line

In [9]:
# Distance to shoreline polygon
bndpoly_line = 'bndpoly_5line'
arcpy.PolygonToLine_management(bndpoly_multi, bndpoly_line)

distSL = 'distSL_bndpoly'
outDistSL = arcpy.sa.EucDistance(bndpoly_line, cell_size=gridsz)
outDistSL.save(distSL)

In [23]:
# Use points to line
dh_line = 'DH_line'
arcpy.PointsToLine_management(DHpts, dh_line, Sort_Field='sort_fld', Close_Line='NO_CLOSE')

dl_line = 'DL_line'
arcpy.PointsToLine_management(DLpts, dl_line, Sort_Field='sort_fld', Close_Line='NO_CLOSE')

<Result '\\\\Mac\\stor\\Projects\\SA_DataProcessing\\GDBs\\2008.gdb\\DL_line'>

Manually edited the lines to not connect the most northern point to the most southern. NO_CLOSE didn't accomplish this because the first point according to the sort_fld is in the middle of the island.

In [24]:
# Distance raster
distDH = 'distDH_DHline'
outDistDH = arcpy.sa.EucDistance(dh_line, cell_size=gridsz)
outDistDH.save(distDH)

# Distance raster
distDL = 'distDL_DLline'
outDistDH = arcpy.sa.EucDistance(dl_line, cell_size=gridsz)
outDistDH.save(distDL)

#### Other attempts

In [271]:
#%% Use my function to create ShoreBetweenInlets
dh_line = 'DH_lineInlets'
CreateShoreBetweenInlets(DHpts, inletLines, dh_line, DHpts, plants_epsg, extend_dist=gridsz, sort_fld='sort_fld')
# This didn't do any better than Points to Line. 

#%% Threshold distance: Only calculate distance within certain distance of dune point
dune_dist_threshold = 300
dhpts_buff = 'DHpts_buff'
arcpy.Buffer_analysis(DHpts, dhpts_buff, dune_dist_threshold, dissolve_option='ALL')
arcpy.env.mask = dhpts_buff # Use points buffer as mask

# Distance to points with threshold
distDH = 'distDH_ptsBuff'
outDistDH = arcpy.sa.EucDistance(DHpts, cell_size=gridsz)
outDistDH.save(distDH)

# Distance to line with threshold
distDH = 'distDH_line_ptsBuff'
outDistDH = arcpy.sa.EucDistance(dh_line, cell_size=gridsz)
outDistDH.save(distDH)

# Reset mask
arcpy.env.mask = bndpoly_multi

## Points

Processing notes:

- "object" type columns won't be included in the output FC.
- We don't want to use Boolean variables because they don't accept NaN values. 
- Using the true_values and false_values arguments in read_excel doesn't always work for Ungulate_Grazed. 
- So I convert them to numeric by just recoding them with replace.

In [198]:
# Set up spreadsheet import
plants_xls = r'\\Mac\stor\Projects\SA_DataProcessing\input_data\WILD_AMPU_COMBO_01to18.xlsx'
colnames = ['YEAR', 'ID', 'NORTHING', 'EASTING', 'GPS_Height', 'AREA_cm2', 'Caged', 'Ungulate_Grazed', 'Insect_Grazed', 'Grazed']
xlscols = 'A:G,K:N' # Select columns in xls file: Year to area and caged to grazed

In [199]:
# Import spreadsheet as pandas DF and subset to the given year
allplants = pd.read_excel(plants_xls, index_col='New ID', parse_cols=xlscols, names=colnames) 
plants = allplants[allplants.YEAR == year]

# Convert variables to numeric by coding values for grazed status
grazed_codes = {'Y':1, 'N':0, ' N':0, 'D':2, 'H':3, 'I':4, 'HI':5, 'DI':6, 'U':7, 'UU':8, 'UUI':9, 'D,I':24, r'N/A':fill}
plants = plants.replace(grazed_codes)
plants = plants.fillna(fill)

# QC the columns
print('Unique values in given columns:')
for c in ['Caged', 'Insect_Grazed', 'Ungulate_Grazed', 'Grazed']:
    print('{}: {}'.format(c, plants[c].unique()))
print('\nColumn data types:')
print(plants.dtypes)
print('\nFirst five rows:')
plants.head()

Unique values in given columns:
Caged: [1 0]
Insect_Grazed: [  1.00000000e+00   0.00000000e+00  -9.99990000e+04]
Ungulate_Grazed: [0 1]
Grazed: [ 4  7  2  9  8  6 24  3  1]

Column data types:
YEAR                 int64
ID                  object
NORTHING           float64
EASTING            float64
GPS_Height         float64
AREA_cm2           float64
Caged                int64
Ungulate_Grazed      int64
Insect_Grazed      float64
Grazed               int64
dtype: object

First five rows:


,YEAR,ID,NORTHING,EASTING,GPS_Height,AREA_cm2,Caged,Ungulate_Grazed,Insect_Grazed,Grazed
New ID,,,,,,,,,,
7210,2008,a1,4239865.685,490662.512,-35.557,45.0,1,0,1.0,4
7211,2008,a10,4217187.966,483176.644,-35.113,30.0,0,0,0.0,7
7212,2008,a100,4232946.369,488211.500,-36.646,5.0,0,0,1.0,4
7213,2008,a101,4232720.250,488208.124,-35.673,30.0,0,0,1.0,4
7214,2008,a102,4214407.771,481845.987,-36.823,5.0,1,0,1.0,4


#### Investigate duplicate points.

- In 2008, there are 5 coordinates pairs that have many duplicate entries. All the duplicates have fill values for GPS_Height. They have different AREA_cm2 values.
- The ID value and coordinates (NORTHING, EASTING) are completely correlated in 2008, but not in other years. 
- Years with no duplicate coordinate pairs: 2003, 2010-2018

In [201]:
# QC: check for entries at the same point
print("Total number of plant points: {}".format(len(plants)))
print("Number of unique ID values among plants: {}".format(len(plants['ID'].unique())))

duplicates = plants.loc[plants.duplicated(['NORTHING', 'EASTING'], keep=False)]
print("Total number of duplicates: {}".format(len(duplicates)))

print('\nNumber of unique duplicated values by field:')
mapping = {}
for col in colnames:
    mapping[col] = len(duplicates[col].unique())
print(pd.Series(data=mapping))
print('\n')

unique_duplicates = duplicates.drop_duplicates()
print("Number of unique entries among duplicates: {}".format(len(unique_duplicates)))
unique_duplicates = duplicates.drop_duplicates(['NORTHING', 'EASTING'])
print("Number of unique XY entries among duplicates: {}".format(len(unique_duplicates)))

# duplicates.groupby('ID').groups

Total number of plant points: 1048
Number of unique ID values among plants: 787
Total number of duplicates: 266

Number of unique duplicated values by field:
AREA_cm2           30
Caged               2
EASTING             5
GPS_Height          1
Grazed              2
ID                  5
Insect_Grazed       2
NORTHING            5
Ungulate_Grazed     2
YEAR                1
dtype: int64


Number of unique entries among duplicates: 95
Number of unique XY entries among duplicates: 5


In [202]:
# QC: check for entries at the same point
duplicates = plants.loc[plants.duplicated(['NORTHING', 'EASTING'], keep=False)]

print("Total number of duplicates (XY): {}".format(len(duplicates)))
print("Number of unique duplicated values (XY): {}".format(len(duplicates['AREA_cm2'].unique())))

unique_duplicates = duplicates.drop_duplicates()
print("Number of unique entries among duplicates: {}".format(len(unique_duplicates)))
unique_duplicates = duplicates.drop_duplicates(['NORTHING', 'EASTING'])
print("Number of unique XY entries among duplicates: {}".format(len(unique_duplicates)))

Total number of duplicates (XY): 266
Number of unique duplicated values (XY): 30
Number of unique entries among duplicates: 95
Number of unique XY entries among duplicates: 5


#### GPS_Height

All duplicates have fill in GPS_Height. Is this a flag for duplicated entries? 

- Using GIS, I see that there are 5 sites where GPS_Height values are fill.Those are all duplicated with the following quantities (from south to north): 104, 32, 52, 52, and 26.
- After removing the duplicate rows, there are still 95 fill values in GPS_Height.

In [203]:
# Look at GPS_Height in duplicates
print(duplicates['GPS_Height'].min())
print(duplicates['GPS_Height'].max())
all(duplicates['GPS_Height'] == fill)

-99999.0
-99999.0


True

In [204]:
# Look at GPS_Height without duplicates
plants_nodups = plants.drop_duplicates()
print(plants_nodups['GPS_Height'].min())
print(plants_nodups['GPS_Height'].max())
print(all(plants_nodups['GPS_Height'] == fill))
print(sum(plants_nodups['GPS_Height'] == fill))
plants_nodups.loc[plants_nodups['GPS_Height'] == fill].head()

-99999.0
-31.883
False
95


,YEAR,ID,NORTHING,EASTING,GPS_Height,AREA_cm2,Caged,Ungulate_Grazed,Insect_Grazed,Grazed
New ID,,,,,,,,,,
7992,2008,Area_a1,4215483.0,482405.0,-99999.0,1.0,0,0,0.0,7
7993,2008,Area_a1,4215483.0,482405.0,-99999.0,2.0,0,0,-99999.0,7
7994,2008,Area_a1,4215483.0,482405.0,-99999.0,1.0,0,0,-99999.0,7
8024,2008,Area_a2,4218607.0,483755.0,-99999.0,1.0,0,1,-99999.0,1
8025,2008,Area_a2,4218607.0,483755.0,-99999.0,2.0,0,1,-99999.0,1


#### AREA_cm2

The duplicates have different AREA values. Does this mean they're not duplicate entries? 

- Does the difference in AREA indicate that a plant was surveyed multiple times over the course of the year and each time it may have grown? 

In [87]:
a1_dups = duplicates.loc[duplicates['ID']=='Area_a1']
a1_dups

print(a1_dups['AREA_cm2'].min())
print(a1_dups['AREA_cm2'].max())
print(len(a1_dups))

a1_dups.head()

1.0
2.0
31


,YEAR,ID,NORTHING,EASTING,GPS_Height,AREA_cm2,Caged,Ungulate_Grazed,Insect_Grazed,Grazed
New ID,,,,,,,,,,
7993,2008,Area_a1,4215483.0,482405.0,-99999.0,2.0,0,0,-99999.0,7
7994,2008,Area_a1,4215483.0,482405.0,-99999.0,1.0,0,0,-99999.0,7
7995,2008,Area_a1,4215483.0,482405.0,-99999.0,2.0,0,0,-99999.0,7
7996,2008,Area_a1,4215483.0,482405.0,-99999.0,1.0,0,0,-99999.0,7
7997,2008,Area_a1,4215483.0,482405.0,-99999.0,2.0,0,0,-99999.0,7


In [ ]:
grouped_dups = duplicates.groupby(['NORTHING', 'EASTING'])
# for name, group in grouped_dups:
#     print(name)
#     print(group)

# grouped_dups.first()
grouped_dups.count()
print(grouped_dups.min())
print(grouped_dups.max())

### Proceed with processing

In [66]:
# Add coordinates latlon and xy (NAD83 UTM 18N) 

# Use pyproj to convert projected coordinates to geographic coordinates (lat, lon in NAD83)
lon, lat = pyproj.transform(pyproj.Proj(init='epsg:{}'.format(plants_epsg)), 
                            pyproj.Proj(init='epsg:4269'), # NAD83
                            plants['EASTING'].tolist(), 
                            plants['NORTHING'].tolist())
plants['lon_nad83'] = lon
plants['lat_nad83'] = lat

print('\nFirst five rows:')
plants.head()


First five rows:


,YEAR,ID,NORTHING,EASTING,GPS_Height,AREA_cm2,Caged,Ungulate_Grazed,Insect_Grazed,Grazed,lon_nad83,lat_nad83
New ID,,,,,,,,,,,,
7210,2008,a1,4239865.685,490662.512,-35.557,45.0,1,0,1.0,4,-75.106799,38.306839
7211,2008,a10,4217187.966,483176.644,-35.113,30.0,0,0,0.0,7,-75.191883,38.102346
7212,2008,a100,4232946.369,488211.500,-36.646,5.0,0,0,1.0,4,-75.134718,38.244450
7213,2008,a101,4232720.250,488208.124,-35.673,30.0,0,0,1.0,4,-75.134753,38.242412
7214,2008,a102,4214407.771,481845.987,-36.823,5.0,1,0,1.0,4,-75.206989,38.077263


In [49]:
# Create points FC for given year
DFtoFC(plants, os.path.join(arcpy.env.workspace, plants_outfc), spatial_ref=crs, xy=['EASTING', 'NORTHING'], keep_fields='all', fill=fill)

# Report point count
plant_ct = int(arcpy.GetCount_management(plants_outfc)[0])
print('Created plants FC with {} points: {}'.format(plant_ct, plants_outfc))
print()
print('Fields in {}:'.format(plants_outfc))
for f in arcpy.ListFields(plants_outfc):
    print('   {}'.format(f.name))

Created plants FC with 1048 points: pts_plants

Fields in pts_plants:
   OBJECTID
   Shape
   New_ID
   YEAR
   GPS_Height
   AREA_cm2
   Caged
   Ungulate_Grazed
   Insect_Grazed
   Grazed


In [ ]:
# Create training set with random points and plant observations. 
# Distribute random points within shoreline polygon. Number of randoms is proportional to number of plants. Points will not be closer than the analysis distance (gridsz).
randoms_ct = ranPts_per_PlantPt * plant_ct
arcpy.CreateRandomPoints_management(yeargdb, ranPts, bndpoly_multi, number_of_points_or_field=randoms_ct, minimum_allowed_distance=gridsz)

In [50]:
# Merge random points and plant points to make training set.
arcpy.Merge_management([plants_outfc, ranPts], trainingPts)

# QC
print('Plants points count: {}'.format(plant_ct))
print('Random points count: {}'.format(arcpy.GetCount_management(ranPts)[0]))
print('Training points count: {} ({})'.format(arcpy.GetCount_management(trainingPts)[0], trainingPts))
print()
print('Fields in {}:'.format(trainingPts))
for f in arcpy.ListFields(trainingPts):
    print('   {}'.format(f.name))

Plants points count: 1048
Random points count: 3144
Training points count: 4192 (pts_training)

Fields in pts_training:
   OBJECTID
   Shape
   New_ID
   YEAR
   GPS_Height
   AREA_cm2
   Caged
   Ungulate_Grazed
   Insect_Grazed
   Grazed
   CID


#### Get plant points for previous year

from Ben: add 2 fields that can serve as a metric for how close a site is to plants from previous year (do for both plants and randoms):

a. number of plants from previous year (2007 in this case) in X-meter radius?- lets try 30 m

b. distance to nearest plant from previous year? - if there is a way to define an along barrier axis and have a positive value with the distance is updrift (northward of the point), and negative if it's south/downdrift.

In [13]:
# Subset to the prior year
lastyears_plants = allplants[allplants.YEAR == year-1]

# Create points FC for given year
lastyear_pts = '{}{}'.format(plants_outfc, year-1)
DFtoFC(lastyears_plants, os.path.join(arcpy.env.workspace, lastyear_pts), spatial_ref=crs, xy=['EASTING', 'NORTHING'], fill=fill)

print('OUTPUT: {}'.format(pts_lastyears))

... converting dataframe to array... 
... converting array to feature class... 

OUTPUT: pts_plants2007


In [19]:
# From above:
# Set Snap Raster environment. All subsequent rasters will align to these cells.
arcpy.env.snapRaster = DEMres
arcpy.env.extent = barrierBoundary

#### Use PointStatistics to make raster with count of points within 30 radius

In [233]:
arcpy.AddField_management(lastyear_pts, 'cnt', 'SHORT', '1')
for f in arcpy.ListFields(lastyear_pts):
    print(f.name)
    
with arcpy.da.UpdateCursor(lastyear_pts, ['cnt']) as cursor:
    for row in cursor:
        cursor.updateRow([1])

In [234]:
# Count points in 30 m radius of cell
lycnt_raster = 'ptcnt{}_{}m'.format(year-1, lypd_radius)
outRas = arcpy.sa.PointStatistics(lastyear_pts, 'cnt', neighborhood=arcpy.sa.NbrCircle(lypd_radius, 'MAP'), statistics_type='SUM')
outRas.save(lycnt_raster)

## Extract raster values to points

In [46]:
distSL = 'distSL_SLline'
distDL = 'distDL'
distDH = 'distDH'
aspect = '{}_aspect'.format(DEMres)
slope = '{}_slope'.format(DEMres)

In [51]:
# Extract values from rasters to training points
arcpy.sa.ExtractMultiValuesToPoints(trainingPts, [[DEMres, 'elev'], [aspect, 'aspect'], [slope, 'slope'], 
                                                  [distSL, 'dist_sl'], [distDH, 'dist_dh'], [distDL, 'dist_dl'], 
                                                  [vegRaster, 'veg_type']])

print('Fields in {} after ExtractMultiValuesToPoints():'.format(trainingPts))
for f in arcpy.ListFields(trainingPts):
    print('   {}'.format(f.name))
# Audio(filename=sound_file, autoplay=True) # Play sound when cell is complete.

Fields in pts_training after ExtractMultiValuesToPoints():
   OBJECTID
   Shape
   New_ID
   YEAR
   GPS_Height
   AREA_cm2
   Caged
   Ungulate_Grazed
   Insect_Grazed
   Grazed
   CID
   elev
   aspect
   slope
   dist_sl
   dist_dh
   dist_dl
   veg_type


### Convert FC to DF and add coordinates

In [72]:
# Convert to DF
trainingpts_df = FCtoDF(trainingPts, xy=True)
trainingpts_df.head()

Converting feature class to array with X and Y...
...converting array to dataframe...


In [73]:
# Rename XY columns and add latlon
xcol = 'x_utm'
ycol = 'y_utm'
trainingpts_df.rename(columns={'SHAPE@X':xcol,'SHAPE@Y':ycol}, inplace=True)

# Use pyproj to convert projected coordinates to geographic coordinates (lat, lon in NAD83)
lon, lat = pyproj.transform(pyproj.Proj(init='epsg:{}'.format(plants_epsg)), 
                            pyproj.Proj(init='epsg:4269'), # NAD83
                            trainingpts_df[xcol].tolist(), 
                            trainingpts_df[ycol].tolist())
trainingpts_df['lon_nad83'] = lon
trainingpts_df['lat_nad83'] = lat

trainingpts_df.head()

## Save as xls

Here are two options (tried and failed to use ArcPy to create an Excel file). One uses converts the Pandas dataframe to an Excel file and the other manually exports the Excel file from ArcGIS.

In [83]:
# Convert dataframe to Excel file
pts_outxls = os.path.join(os.path.dirname(home), 'outputs', 'training{}_v2.xls'.format(year))
trainingpts_df.to_excel(pts_outxls, na_rep=fill, index=False)

print("OUTPUT: {}".format(pts_outxls))

OUTPUT: \\Mac\stor\Projects\SA_DataProcessing\outputs\training2008_v2.xls


I ran this tool in ArcGIS Pro from the Geoprocessing pane and it worked great. This is the Python command it claims to run. When I tried running the command it fails with "AttributeError: DescribeData: Method fields does not exist"

```python
arcpy.conversion.TableToExcel("pts_training", 
                              r"X:\Projects\SA_DataProcessing\outputs\pts_training_TableToExcel2.xls", 
                              "NAME", 
                              "CODE")
```